In [233]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from tokenizers import Tokenizer
from tokenizers.models import WordPiece
from tokenizers.trainers import WordPieceTrainer
from tokenizers.normalizers import Lowercase
from tokenizers.pre_tokenizers import Whitespace
from transformers import AutoTokenizer

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


290183
['Rock' 'Metal' 'Pop' 'Indie' 'R&B' 'Folk' 'Electronic' 'Jazz' 'Hip-Hop'
 'Country']


In [8]:
import kagglehub
path = kagglehub.dataset_download("mateibejan/multilingual-lyrics-for-genre-classification")
print(path)

C:\Users\desta\.cache\kagglehub\datasets\mateibejan\multilingual-lyrics-for-genre-classification\versions\1


In [ ]:
# !!! Execute this cell to load the dataset if it is not already loaded

# import pandas as pd
# modified_path = path.replace("C:\\", "/").replace("\\", "/") + "/train.csv"
# df = pd.read_csv(modified_path)

,Artist,Song,Genre,Language,Lyrics
0,12 stones,world so cold,Rock,en,"It starts with pain, followed by hate\nFueled ..."
1,12 stones,broken,Rock,en,Freedom!\nAlone again again alone\nPatiently w...
2,12 stones,3 leaf loser,Rock,en,"Biting the hand that feeds you, lying to the v..."
3,12 stones,anthem for the underdog,Rock,en,You say you know just who I am\nBut you can't ...
4,12 stones,adrenaline,Rock,en,My heart is beating faster can't control these...
...,...,...,...,...,...
290178,bobby womack,i wish he didn t trust me so much,R&B,en,I'm the best friend he's got I'd give him the ...
290179,bad boys blue,i totally miss you,Pop,en,"Bad Boys Blue ""I Totally Miss You"" I did you w..."
290180,celine dion,sorry for love,Pop,en,Forgive me for the things That I never said to...
290181,dan bern,cure for aids,Indie,en,The day they found a cure for AIDS The day the...


In [ ]:
df = pd.read_csv("lyrics_genre.csv")
df.head()
print(len(df))
print(df.Genre.unique())

In [234]:
df = df[df['Language']=='en']
columns_to_drop = ['Language']
df = df.drop(columns=columns_to_drop) 

df = df.groupby(['Artist', 'Song']).agg({
    'Lyrics': 'first',  # Take the first instance of the lyrics for each group
    'Genre': lambda x: list(set(x))       # Aggregate all genres into a list
}).reset_index()
print(len(df))

210010


In [235]:
multi_genre_songs = df[df['Genre'].apply(len) > 1]

# Display these songs
print(multi_genre_songs.head())

       Artist          Song  \
122  2 chainz    100 joints   
169  2 chainz         kesha   
713   50 cent  all his love   
714   50 cent     all of me   
726   50 cent      bad news   

                                                Lyrics           Genre  
122  [Hook]\nNo matter what they say I smoke 100 jo...  [Hip-Hop, Pop]  
169  I am in love, with what we are, not what we sh...  [Hip-Hop, Pop]  
713  [Chorus: singing]\nHe gon' give you all his lo...  [Hip-Hop, Pop]  
714  Mary~\nnow if i give ya all of me what ya gon ...  [Hip-Hop, Pop]  
726  Lloyd Banks in the house, bad news\nTony Yayo ...  [Hip-Hop, Pop]  


In [236]:
columns_to_drop = ['Artist',"Song"]
df = df.drop(columns=columns_to_drop) 


In [237]:
import re
# to reduce vocab size let s try to remove more unecessary symbols and parts 
# Define a cleaning function
def clean_lyrics(lyrics):
    # Remove section tags like [Intro], [Verse 1], etc.
    cleaned = re.sub(r'\[.*?\]', '', lyrics)
    
    # Remove credits or text after "---"
    cleaned = re.split(r'---', cleaned)[0]
    
    # Remove symbols except for line breaks (\n) and alphanumeric characters
    cleaned = re.sub(r'[^\w\s\n]', '', cleaned)
    
    # Remove extra whitespace and blank lines
    cleaned = re.sub(r'\n\s*\n', '\n', cleaned).strip()
    
    return cleaned

# Apply the cleaning function
df['Lyrics'] = df['Lyrics'].apply(clean_lyrics)
print(df['Lyrics'].head(5))

0    Dont call me horse Im a unicorn Mystical magic...
1    Hazy Days With Lazy Ways You get less done but...
2    The summers were longer they went on forever O...
3    You promised me a life You promised me everyth...
4    I looked in the mirror What did you see lookin...
Name: Lyrics, dtype: object


In [238]:
model_name = "bert-base-uncased" 

tokenizer = AutoTokenizer.from_pretrained(model_name)

lyrics = list(df["Lyrics"])

# Applica la tokenizzazione
inputs = tokenizer(lyrics, return_tensors="pt", padding=True, truncation=True, max_length=100)

print(inputs)

texts_ids = pd.DataFrame(inputs.input_ids)

ImportError: Unable to convert output to PyTorch tensors format, PyTorch is not installed.